### Setup - Installations and Imports 

In [96]:
!pip install tensorflow==2.4.1 -U

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [97]:
import tensorflow as tf
import sagemaker
from sagemaker import get_execution_role
from sagemaker import Session
from sagemaker.tensorflow.serving import TensorFlowModel
import numpy as np

In [98]:
sagemaker.__version__

'2.101.1'

## Configuring AWS Environemnt 

In [99]:
role = get_execution_role()

In [100]:
sess = sagemaker.Session()

In [101]:
bucket = sess.default_bucket()

In [103]:
bucket

'sagemaker-ap-south-1-557255531557'

### s3 Upload of autoencoder SavedModel Format Tar File 

In [104]:
model_data = sess.upload_data( path = 'auto_encoder_model.tar.gz.', key_prefix = 'autoencoder')

In [105]:
model_data

's3://sagemaker-ap-south-1-557255531557/autoencoder/auto_encoder_model.tar.gz.'

## Configuring Model and Creating SageMaker Enpoint

### inference.py handles and preprocesses the incoming instance for prediction using grpc protocol provided by tensorflow serving

In [114]:
model = TensorFlowModel( source_dir = 'code', entry_point='inference.py', model_data=model_data, role = role , framework_version='2.4.1')

In [115]:
predictor = model.deploy( initial_instance_count = 1, instance_type='ml.m4.xlarge', )

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


------!

In [119]:
predictor.endpoint

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


'tensorflow-inference-2022-08-08-18-15-30-227'

### Testing the endpoint

In [120]:
payload = np.array([0.6237936375551048, 0.6126622676499771, 0.8134896393042048, 0.8077562326869803]).reshape(-1,1,4)

In [122]:
import boto3
# kitten_image = open('kitten.jpg', 'rb').read()
endpoint_name = predictor.endpoint_name
runtime_client = boto3.client('runtime.sagemaker')
response = runtime_client.invoke_endpoint(EndpointName=endpoint_name, 
                                   ContentType="application/npy", 
                                   Body=payload.tobytes())
result = response['Body'].read().decode('ascii')

In [123]:
result

'{"predictions": [[[0.5170357823371887, 0.521856963634491, 0.39374643564224243, 0.41626372933387756]]]}'

### The sagemaker endpoint is exposed as a REST Api using the AWS Lambda function and API Gateway

#### The script created below is used to deploy a serverless aws lambda fucntion

In [128]:
%%writefile lambda_function.py


import os
import boto3
import json
import numpy as np

# grab environment variables
ENDPOINT_NAME = os.environ['ENDPOINT_NAME']
runtime= boto3.client('runtime.sagemaker')

def lambda_handler(event, context):
    print("Received event: " + json.dumps(event, indent=2))
    mae_threshold = 0.275  #Mean absolute error Values higher than this is classified as an anomaly
    
    data = json.loads(json.dumps(event))
    payload = data['data']
    payload = np.array(payload)
    print(payload)
    response = runtime.invoke_endpoint(EndpointName=ENDPOINT_NAME,
                                       ContentType="application/npy",
                                       Body=payload.tobytes())

    response = json.loads(response['Body'].read().decode())
    print(response)
    prediction = np.array(response['predictions'])
    
    mae = float(np.mean(np.abs( payload - prediction)))
    anomaly = mae > mae_threshold
    response_dict = {'mae ':mae, 'anomaly ': anomaly}
    
    
    return json.dumps(response_dict)

Writing lambda_function.py
